In [1]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.optimizers import Adam
import matplotlib.pyplot as plt
import numpy as np

In [2]:
from Data.preprocessing import DataPreprocessor
from Models.first_cnn_topology import create_first_cnn_model
from Models.second_cnn_topology import create_second_cnn_model

In [3]:
# Step 2: Initialize Preprocessor
preprocessor = DataPreprocessor(transform_type='fft', file_path="../Data/musicnet.npz", metadata_path="../Data/musicnet_metadata.csv")

# Step 3: Load Data
X, y = preprocessor.load_data()

Loading file 1 of 10
Loading file 2 of 10
Loading file 3 of 10
Loading file 4 of 10
Loading file 5 of 10
Loading file 6 of 10
Loading file 7 of 10
Loading file 8 of 10
Loading file 9 of 10
Loading file 10 of 10


In [21]:
def partition_data(data, window_size):
    """
    Partition data into smaller segments of a given window size, padding if necessary.
    :param data: List of 2D arrays.
    :param window_size: The size of the window to partition each array.
    :return: A new array with the partitioned data.
    """
    partitioned_data = []
    for array in data:
        # Calculate the number of partitions for the current array
        num_partitions = array.shape[1] // window_size
        # Slice the array into partitions and append to the list
        for i in range(num_partitions):
            start = i * window_size
            end = start + window_size
            partitioned_data.append(array[:, start:end])
        # Check if the last partition is smaller than the window_size
        if array.shape[1] % window_size != 0:
            # Pad the last partition with zeros
            last_partition = array[:, -window_size:]
            padding_width = window_size - last_partition.shape[1]
            padding = np.zeros((array.shape[0], padding_width))
            last_partition_padded = np.concatenate((last_partition, padding), axis=1)
            partitioned_data.append(last_partition_padded)
    # Stack all the partitions along a new axis
    partitioned_data = np.stack(partitioned_data, axis=0)
    return partitioned_data

# Partition X and y with the modified function
window_size = 10
X_partitioned = partition_data(X, window_size)
y_partitioned = partition_data(y, window_size)

# Add a channel dimension to X
X_partitioned = X_partitioned[..., np.newaxis]
# Reshape y to be 2D
y_partitioned = y_partitioned.reshape(y_partitioned.shape[0], -1)

print(f"Partitioned X shape: {X_partitioned.shape}")
print(f"Partitioned y shape: {y_partitioned.shape}")


Partitioned X shape: (33218, 2048, 10, 1)
Partitioned y shape: (33218, 990)


In [22]:
# Step 5: Split Data
X_train, X_test, y_train, y_test = train_test_split(X_partitioned, y_partitioned, test_size=0.2, random_state=42)

In [23]:
# Step 6: Define Models
input_shape = X_train.shape[1:]  # Assuming X_train is already in the correct shape
num_classes = y_train.shape[1]   # Assuming y_train is one-hot encoded

In [24]:
model_1 = create_first_cnn_model(input_shape, num_classes)
model_2 = create_second_cnn_model(input_shape, num_classes)

In [25]:
# Step 7: Compile Models
optimizer = Adam()
model_1.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])
model_2.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
# Step 8: Train Models
history_1 = model_1.fit(X_train, y_train, epochs=10, validation_split=0.1)
history_2 = model_2.fit(X_train, y_train, epochs=10, validation_split=0.1)

Epoch 1/10
748/748 [==============================] - 21s 27ms/step - loss: 27683262464.0000 - accuracy: 0.0195 - val_loss: 109550575616.0000 - val_accuracy: 0.0000e+00
Epoch 2/10
748/748 [==============================] - 20s 26ms/step - loss: 705578270720.0000 - accuracy: 0.0222 - val_loss: 1441391181824.0000 - val_accuracy: 0.0000e+00
Epoch 3/10
748/748 [==============================] - 20s 26ms/step - loss: 3783133233152.0000 - accuracy: 0.0215 - val_loss: 6010495827968.0000 - val_accuracy: 0.0000e+00
Epoch 4/10
748/748 [==============================] - 19s 26ms/step - loss: 11350721953792.0000 - accuracy: 0.0222 - val_loss: 15732888829952.0000 - val_accuracy: 0.0000e+00
Epoch 5/10
748/748 [==============================] - 19s 26ms/step - loss: 25344614596608.0000 - accuracy: 0.0209 - val_loss: 32607565774848.0000 - val_accuracy: 0.0000e+00
Epoch 6/10
748/748 [==============================] - 91s 122ms/step - loss: 47922162958336.0000 - accuracy: 0.0214 - val_loss: 581703659683

In [ ]:
# Step 9: Evaluate Models
score_1 = model_1.evaluate(X_test, y_test, verbose=0)
score_2 = model_2.evaluate(X_test, y_test, verbose=0)

# Step 10: Compare Results
print(f"Model 1 - Loss: {score_1[0]}, Accuracy: {score_1[1]}")
print(f"Model 2 - Loss: {score_2[0]}, Accuracy: {score_2[1]}")

In [ ]:
# Plot the training history for both models
plt.plot(history_1.history['accuracy'], label='Model 1 Accuracy')
plt.plot(history_1.history['val_accuracy'], label='Model 1 Validation Accuracy')
plt.plot(history_2.history['accuracy'], label='Model 2 Accuracy')
plt.plot(history_2.history['val_accuracy'], label='Model 2 Validation Accuracy')
plt.title('Model Training History')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend()
plt.show()
